In [4]:
import numpy as np

In [5]:
# load the data

K = np.loadtxt('ncicancer_kernel_hf_ex0.txt')
y = np.loadtxt('y1.txt')

In [6]:
K.shape, y.shape

((2305, 2305), (2305,))

In [7]:
y

array([ 47.,  54.,  50., ...,  90.,  76.,  79.])

In [8]:
from sklearn.cross_validation import ShuffleSplit

In [10]:
cv = ShuffleSplit(K.shape[0], n_iter=5, test_size=500, random_state=42)

In [45]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.metrics.regression import r2_score

models = KernelRidge, SVR

scores = {model.__name__: [] for model in models}

for outer_train, outer_test in cv:
    for model in models:
        est = model(kernel='precomputed')
        K_train_train = K[outer_train[:, np.newaxis], outer_train]
        y_train = y[outer_train]
        
        est.fit(K_train_train, y_train)
        K_test_train = K[outer_test[:, np.newaxis], outer_train]
        y_test = y[outer_test]
        pred = est.predict(K_test_train)
        score = r2_score(y_test, pred)
        scores[model.__name__].append(score)

In [50]:
{key: (np.mean(value), np.std(value)) for key, value in scores.items()}

{'KernelRidge': (0.34823307447592955, 0.044513698294497035),
 'SVR': (0.21432798313133766, 0.014427004325354193)}

In [59]:
alphas = np.logspace(-3, 3, 20)
Cs = np.logspace(3, -3, 20)

models = (KernelRidge, 'alpha', alphas), (SVR, 'C', Cs)


In [58]:
from sklearn.cross_validation import KFold


In [79]:
scores = {model[0].__name__: [] for model in models}

for outer_train, outer_test in cv:
    inner_cv = KFold(len(outer_train), n_folds=3)
    for model, param, param_vals in models:
        print("evaluating {}".format(model.__name__))
        inner_scores = []
        K_train_train = K[outer_train[:, np.newaxis], outer_train]
        y_train = y[outer_train]
        K_test_train = K[outer_test[:, np.newaxis], outer_train]
        y_test = y[outer_test]
        for param_val in param_vals:
            #print("testing param {}".format(param_val))
            param_scores = []
            est = model(kernel='precomputed')
            setattr(est, param, param_val)
            for inner_train, inner_test in inner_cv:
                est.fit(K_train_train[inner_train[:, np.newaxis],
                                      inner_train],
                        y_train[inner_train])
                pred = est.predict(K_train_train[
                            inner_test[:, np.newaxis],
                            inner_train])
                score = r2_score(y_train[inner_test], pred)
                param_scores.append(score)
            inner_scores.append(param_scores)
        mean_inner_scores = np.mean(inner_scores, axis=1)
        argbest_param = np.argmax(mean_inner_scores)
        best_param = param_vals[argbest_param]
        print("Best parameter for {}: {}".format(model.__name__, best_param))
        setattr(est, param, best_param)
        est.fit(K_train_train, y_train)
        pred = est.predict(K_test_train)
        score = r2_score(y_test, pred)
        scores[model.__name__].append(score)

evaluating KernelRidge
Best parameter for KernelRidge: 0.162377673919
evaluating SVR
Best parameter for SVR: 26.3665089873
evaluating KernelRidge
Best parameter for KernelRidge: 0.162377673919
evaluating SVR
Best parameter for SVR: 112.883789168
evaluating KernelRidge
Best parameter for KernelRidge: 0.162377673919
evaluating SVR
Best parameter for SVR: 54.5559478117
evaluating KernelRidge
Best parameter for KernelRidge: 0.162377673919
evaluating SVR
Best parameter for SVR: 54.5559478117
evaluating KernelRidge
Best parameter for KernelRidge: 0.162377673919
evaluating SVR
Best parameter for SVR: 54.5559478117


In [66]:
scores

{'KernelRidge': [0.44333070663271967,
  0.36307819027440247,
  0.49275022845054484,
  0.33807395840899068,
  0.4050741557000449],
 'SVR': [0.41549767795010839,
  0.39331693768951792,
  0.47944727269104948,
  0.40553377137278235,
  0.42177057985852218]}

In [67]:
{key: np.mean(value) for key, value in scores.items()}

{'KernelRidge': 0.40846144789334049, 'SVR': 0.42311324791239607}